<center>
    <span style="color:blue; font-family:Georgia;  font-size:2em;">
        <h1>Recommender systems based on collaborative filtering and latent factors</h1></span>
 </center>
        <p> </p>
        <p> </p>
        <center><span style="color:blue; font-family:Georgia;  font-size:1em;">
        Ramon Béjar Torres</span></center>
        <canvas id="myCanvas" width="200" height="100" style="border:0px solid"></canvas>
        <center>Data mining - Master on Computer Science</center>
        <center><img src="M-UdL2.png"  width="200" alt="UdL Logo"></center>

We are going to discuss an approach to build a recommender system based on using the whole information stored in the user-product ranking matrix. That is, a matrix where:
        $$ M[i,j]  = \left\{ \begin{matrix}  \textrm{ranking  information  about  user  i  and  product  j} \\ Nan \ \textrm{if  such ranking is not yet known} \end{matrix} \right. $$
        
We assume that such matrix gives **explicit rankings** provided by users to the products they have bought.

Because we use the whole information in such matrix to predict unknown entries, we say that we follow a collaborative (or global) filtering approach.

The approach based on latent factors, is based on two complementary assumptions:
1. User i can be encoded as a vector of c feature values, that encode the preferences of the user for different categories/kinds of products.
2. Product j can also be encoded as a vector of c feature values, that encode the weight that the product has in each one of these categories.

So, how much an user likes a product will be measured as the **dot product** of these two vectors.
$$ U_i \cdot P_j = \sum_k U_{i,k} \cdot P_{j,k} $$


In [1]:
#
# Our preliminary set-up code
#

import pyspark
import os
import math
import random
import sys

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt


spark_home = os.environ.get('SPARK_HOME', None)
sc = pyspark.SparkContext('local[*]')

print (spark_home)
sc

/usr/local/share/spark-3.0.1-bin-hadoop2.7


<SparkContext master=local[*] appName=pyspark-shell>

## U-P matrix factorization for latent factors discovery

The approach to discover latent factors that characterize users and products, is the one based on U-P matrix factorization. The problem formulation is that we want to find a factorization of our user-product matrix as the product of two matrices $U$ and $V$, where:
1. Each user will be represented as a row vector with $c$ factors in matrix $U$, so $U$ will be a matrix with $m$ rows and $c$ columns.
2. Each product will be represented as a column vector with $c$ factors in matrix $P$, so $P$ will be a matrix with $c$ rows and $n$ columns.


Then, given our user-product rating matrix, with $m$ rows (users) and $n$ columns (products) we want to find matrices $U$ and $P$ such:

$$ \overbrace{ \left( \begin{matrix}
  U_{1,1} & \cdots & U_{1,c} \\
   & \cdots & \\
  & \vdots & \\
  U_{m,1} & \cdots & U_{m,c}
 \end{matrix} \right) }^{U \ (m\times c)} \times
 \overbrace{ \left( \begin{matrix}
  P_{1,1} & \cdots & P_{1,n} \\
   & \cdots & \\
  & \vdots & \\
  P_{c,1} & \cdots & P_{c,n}
 \end{matrix}  \right)  }^{P \ (c\times n) } \ = \  
  \overbrace{ \left( \begin{matrix}
  M_{1,1} & \cdots & M_{1,n} \\
   & \cdots & \\
  & \vdots & \\
  M_{m,1} & \cdots & P_{m,n}
 \end{matrix}  \right)  }^{M (m \times n)}$$


Remember that in general the matrix $M$ may have empty entries, but our U-P factorization will provide values for all the entries, so when we say an U-P factorization is good for a partially filled matrix M, we usually mean that it agrees with the value for the filled entries of M. Once we have this factorization, observe that the value of any entry $(i,j)$ of the matrix M, even for unknown entries, is predicted multiplying the row $i$ of matrix $U$ by column $j$ of matrix P, that is:

$$ \hat{M}(i,j) = \sum_{k=1}^c U_{i,k} * P_{k,j} $$

Because it may be not possible to find such **exact** factorization with the desired number of latent factors, finding such factorization is actually presented as an optimization problem, where the goal is to find a factorization with the smallest RMSE error (error computed over the filled entries of M):

 $$ \textrm{RMSE}(U,P,M) = \sqrt{\frac{\sum_{i,j}(U(row_i) \ \cdot \ P(col_j) - M(i,j))^2}{\# \ known \ entries}} $$
 where the summatory is over entries $(i,j)$ of the matrix M with known values.

However, because we are really more interested in being able to **predict unknown values** of the matrix M, than in predicting the known ones, to avoid **overfitting** to the known values and  have a high error rate for unknown entries, the optimization algorithms usually also allow to consider alternative objective functions that incorporate regularized terms to control the tradeoff between:
- RMSE error
- Prediction of unknown entries.

In general, the regularized terms have the goal to penalize too complex models that are very sensitive to small changes in the input. That is, to penalize models where many values of $row_i$ (user i) or $column_j$ (product j) are away from zero.

## Optimization algorithms for U-P matrix factorization

We may consider different algorithms for finding a good U-P factorization (with a small RMSE error or any other generalized error function). These are the two main ones:
1. Gradient descend: Using the same approach we presentend for finding a linear model, but this time applied to the parameters of our model (the coefficients of the matrices U and P)

2. Alternating Least Squares: If we keep fixed one of the two matrices (U or P), the problem becomes a quadratic optimization problem that can be optimally solved in P-time. So, we perform an iterative process where we fix one of them, find the optimal one for the other, and in the next iteration we exchange the roles: the second one is fixed and the first one is optimized. This process is repeated until we reach a fixed point.



To know more about this problems and their solving algorithms, a good source of information is the paper:

> Yehuda Koren, Robert Bell and Chris Volinsky. *Matrix factorization techniques for recommender systems*. In Computer Journal, IEEE press, Vol 42(8), 2009. 
https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf

and also the book about data mining for big data we recommended for this course.

Let's consider the same example matrix we used in our notebook about clustering algorithms. In particular, let's consider a data base of users, where for each user we store the ratings given by the user to different movies. We first consider the matrix full of entries, but later we consider variations of this matrix where some of the entries will be empty.

In [2]:
# Example data
#
# We have 10 users, and 10 movies: STW1, STW2, STW3, STW4, STW5, STW6
#                                  T1, T2, T3 and BaT
# Each entry i,j is the rating given by the user in the range [-5.0,5.0]
# We can observe that we have 4 clear Star Wars fans (that they also like a 
# little bit Terminator movies)
# We also have four clear Terminator fans (that they also like a little STWs movies)
# Finally, we have two clear Breakfast at tiffannies fans (BaT), that they do not
# like too much science-fiction movies

usersandmovies = [ [3,3,3,5,5,4, 3,3,-1, -1], \
                   [3,3,3,5,5,4, 4,2,0, -1], \
                   [3,3,4,5,5,4, 4,4,1, 0], \
                   [4,3,3,4,5,4, 3,3,1, -1], \
                   [1,1,1,0,1,1, 5,4,2, -1], \
                   [1,2,1,0,1,1, 4,4,2, -1], \
                   [1,2,2,1,1,1, 4,4,2, -1], \
                   [1,2,2,1,1,0, 5,4,3, -1], \
                   [-2,-3,-2,0,-2,-1, 0,0,-1,4], \
                   [-2,-3,-2,0,-2,-1, 0,0,-1,4]   ]

In [9]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

# We need a function to convert our matrix format to a RDD of 
# pyspark.mllib.recommendation.Rating objects:
#
#    Rating(int(userid),int(productid),float(rating)
# 
# This function converts a user row to a Rating object: (user_i, product_j, rating_{i_j})

def convertMatrixRowToRating( userRow ):
    rating = []
    for product,productrating in enumerate(userRow[0]):
            rating.append( Rating( userRow[1] , product, float(productrating) ) )
    return rating       

In [55]:
# Let's assume that the users and movies matrix is loaded from a text file
matrixRDD = sc.textFile( "usersandmovies.txt" ).map( lambda x : x.split(',') ).cache()
matrixRDDWithIndex = matrixRDD.zipWithIndex()
# matrixRDDWithIndex.collect()
ratingsRDD = matrixRDDWithIndex.flatMap( lambda x : convertMatrixRowToRating( x ) )
print(ratingsRDD.take(2))

# Build the recommendation model using Alternating Least Squares
rank = 3
numIterations = 10
model = ALS.train(ratingsRDD, rank, numIterations)

[Rating(user=0, product=0, rating=3.0), Rating(user=0, product=1, rating=3.0)]


In [56]:
# Evaluate the model on training data
# First, get the data without the rating values (only user-product IDs)
testdata = ratingsRDD.map(lambda p: (p[0], p[1]))

# Next, Get the predictions obatined with our U-V factorization model
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
# join ((u,p), V) and ((u,p), W) to get ((u,p), (V, W))
ratesAndPreds = ratingsRDD.map( lambda r: ((r[0], r[1]), r[2]) ).join(predictions)

## Compute Mean Square error
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 0.08825841155062698


In [17]:
# We can also get the U-P factorization, as the set of user features (latent factors) of
# the U matrix

print (" Users latent factors:")
for userfactors in model.userFeatures().sortByKey().collect():
    print (userfactors)

 Users latent factors:
(0, array('d', [-1.6163338422775269, -0.800126314163208, 1.6512421369552612]))
(1, array('d', [-1.5290096998214722, -0.8449429869651794, 1.4825485944747925]))
(2, array('d', [-1.1987035274505615, -1.267828106880188, 1.5914820432662964]))
(3, array('d', [-1.462148904800415, -0.7700231075286865, 1.0145211219787598]))
(4, array('d', [0.7024548649787903, -1.1889294385910034, -0.6107591986656189]))
(5, array('d', [0.4982064962387085, -0.9866253733634949, -0.7544174790382385]))
(6, array('d', [0.34337934851646423, -1.0374592542648315, -0.5147237777709961]))
(7, array('d', [0.6376985907554626, -1.2051270008087158, -0.8138543963432312]))
(8, array('d', [1.1655744314193726, -0.6825593709945679, 1.6304762363433838]))
(9, array('d', [1.1655744314193726, -0.6825593709945679, 1.6304762363433838]))


In [18]:
# and the set of product features of the V matrix:
print ("\n Products latent factors:")
for productfactors in model.productFeatures().sortByKey().collect(): 
    print (productfactors)


 Products latent factors:
(0, array('d', [-1.8319659233093262, -1.4858349561691284, -0.5674116015434265]))
(1, array('d', [-2.0352492332458496, -1.8713034391403198, -1.151132583618164]))
(2, array('d', [-1.761841893196106, -1.8466660976409912, -0.6929657459259033]))
(3, array('d', [-1.6989115476608276, -1.6264508962631226, 0.5167595148086548]))
(4, array('d', [-2.4566285610198975, -1.9667993783950806, -0.2951808571815491]))
(5, array('d', [-1.762921929359436, -1.5366034507751465, -0.010525639168918133]))
(6, array('d', [-0.9206563830375671, -3.884936571121216, -0.9814675450325012]))
(7, array('d', [-0.7825227379798889, -3.4255120754241943, -0.8905119299888611]))
(8, array('d', [-0.2589675486087799, -1.5595213174819946, -1.0811312198638916]))
(9, array('d', [1.622470498085022, 0.7265133857727051, 1.5440515279769897]))


Can you identify significant differences between the latent factors for different user groups and product groups ? Observe that the two "Breakfast at Tiffany's" fans have clearly different latent factors than the others users

Let's next check what happens if there are some missing values in the user-product matrix. Consider the following function to randomly erase $k$ entries from each row of the matrix:

## Prediction model with incomplete data

Let's consider now the extraction of some of the elements of the user-product matrix. Consider randomly eliminating k elements from each row.

In [32]:
def convertMatrixRowToRatingWithBlanks( userRow, k ):
    rating = []
    size = len(userRow[0])
    blanks = random.sample(range(size), k)the error can decrease when there are less entries available to fit a good model,
    for product,productrating in enumerate(userRow[0]):
        if (product not in blanks):
            rating.append( Rating( userRow[1] , product, float(productrating) ) )
    return rating       

In [53]:
ratings2RDD = matrixRDDWithIndex.flatMap( lambda x : convertMatrixRowToRatingWithBlanks( x, 4 ) )
print(ratings2RDD.take(10))

# Build the recommendation model using Alternating Least Squares
rank = 3
numIterations = 10
model2 = ALS.train(ratings2RDD, rank, numIterations)

[Rating(user=0, product=0, rating=3.0), Rating(user=0, product=2, rating=3.0), Rating(user=0, product=4, rating=5.0), Rating(user=0, product=5, rating=4.0), Rating(user=0, product=6, rating=3.0), Rating(user=0, product=7, rating=3.0), Rating(user=1, product=0, rating=3.0), Rating(user=1, product=1, rating=3.0), Rating(user=1, product=4, rating=5.0), Rating(user=1, product=5, rating=4.0)]


In [54]:
# Evaluate the model on training data
# First, get the data without the rating values (only user-product IDs)
testdata2 = ratings2RDD.map(lambda p: (p[0], p[1]))

# Next, Get the predictions obatined with our U-V factorization model
predictions2 = model2.predictAll(testdata2).map(lambda r: ((r[0], r[1]), r[2]))
# join ((u,p), V) and ((u,p), W) to get ((u,p), (V, W))
ratesAndPreds2 = ratings2RDD.map( lambda r: ((r[0], r[1]), r[2]) ).join(predictions2)

# Make predictions also for the whole dataset
predictAll = model2.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPredsAll = ratingsRDD.map( lambda r: ((r[0], r[1]), r[2]) ).join(predictAll)

## Compute Mean Square error
MSE2 = ratesAndPreds2.map(lambda r: (r[1][0] - r[1][1])**2).mean()
MSEAll = ratesAndPredsAll.map(lambda r: (r[1][0] - r[1][1])**2).mean()

print("Mean Squared Error = " + str(MSE2) +  " Mean Square Error over complete data: "+str(MSEAll) )

Mean Squared Error = 0.39505781529231515 Mean Square Error over complete data: 0.8024955879665452


Results for different values for k (eliminated entries from each row):

$$ 
\begin{matrix}
 k & MSE & MSE \ \textrm{for all data} \\ \hline
1 & 0.103 & 0.099 \\
2 & 0.110 & 0.115 \\ 
3 & 0.198 & 0.226 \\
4 & 0.395 & 0.802  
\end{matrix} 
$$

But different executions with the same k will give you slightly different results.

So, the model predition on unknown entries can be worse than the ones of kwnown entries when the number of known entries is low. Actually, a problem of this factorization approach is that to be able to get a good factor vector for a given user, we need enough known entries for that user.

In contrast, there are recommender systems for products in on-line stores based on direct product-to-product comparison that allow to give recommendations for users from any single previously bought (or ranked) product of that user. So, even if the user only made ONE purchase the system will be able to give recommendations, as soon as the total number of user purchases in the on-line store is high enough.

## Collaborative Filtering for Implicit Feedback Datasets

We can also consider data where there is no explicit user feedback. That is, when the entry $M(i,j)$ does not contain an explicit rating of the user, but some **aggregation** of values that can be used to infer something about the preference of user i for product j

For example:
- In a web shop, $M(i,j)$ could indicate the number of times that the user i clicked on product j
- In a on-line video shop, $M(i,j)$ could indicate the fraction of watched time for movie $j$ by user $i$
- We could also give more relevance if the movie was watched with less interruptions.

In general, we can set up $M(i,j)$ to represent some aggregation of indicators such that the more inputs we have, the higher the value of $M(i,j)$, and so the higher our confidence with respect to the preference of user $i$ for product $j$.

To know more about the ALS algorithm (and other global filtering algorithms) in spark, check the documentation page:

> https://spark.apache.org/docs/3.0.0/ml-collaborative-filtering.html

To know more about the model used when we assume that the matrix contains implicit feedback data, you can check here one possible model:

> "Collaborative Filtering for Implicit Feedback Datasets". http://yifanhu.net/PUB/cf.pdf